In [7]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import sklearn.model_selection as ms
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import pandas as pd
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge, ElasticNet
pd.set_option('display.max_columns', 500)

# Load the dataset
Ames = pd.read_csv('~/Documents/NYCDSA/Third Project/ML_Ames_Housing/data/Ames_Housing_Price_Data.csv', index_col=0)

# Convert the below numeric features to categorical features
Ames['MSSubClass'] = Ames['MSSubClass'].astype('object')
Ames['YrSold'] = Ames['YrSold'].astype('object')
Ames['MoSold'] = Ames['MoSold'].astype('object')

# Exclude 'PID' and 'SalePrice' from features and specifically handle the 'Electrical' column
numeric_features = Ames.select_dtypes(include=['int64', 'float64']).drop(columns=['PID', 'SalePrice','BsmtFinSF2', 'BsmtUnfSF', 'BsmtFinSF1','1stFlrSF', '2ndFlrSF', 'LowQualFinSF']).columns
categorical_features = Ames.select_dtypes(include=['object']).columns.difference(['Electrical'])
electrical_feature = ['Electrical']

In [8]:
# Helper function to fill 'None' for missing categorical data
def fill_none(X):
    return X.fillna("None")

# Pipeline for 'Electrical': Fill missing value with mode then apply ordinal encoding
electrical_transformer = Pipeline(steps=[
    ('impute_electrical', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop = 'first')),
])

numeric_transformer = Pipeline(steps=[
    ('impute_mean', SimpleImputer(strategy='mean')),
])

# Pipeline for nominal categorical features: Fill missing values with 'None' then apply one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('impute_none', SimpleImputer(strategy='constant', fill_value='None')),  # Using SimpleImputer
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop = 'first'))
])

# Combined preprocessor for numeric, ordinal, nominal, and specific electrical data
preprocessor = ColumnTransformer(
    transformers=[
        ('electrical', electrical_transformer, ['Electrical']),
        ('num', numeric_transformer, numeric_features),
        ('nominal', categorical_transformer, categorical_features)
])

In [9]:
# Apply the preprocessing pipeline to Ames
transformed_data = preprocessor.fit_transform(Ames)

# Generate column names for the one-hot encoded features
onehot_features = preprocessor.named_transformers_['nominal'].named_steps['onehot'].get_feature_names_out()
all_feature_names = ['Electrical'] + list(numeric_features) + list(onehot_features)

In [10]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Target variable
y = Ames['SalePrice']
# All features
X = Ames.drop(columns='SalePrice')
models = LinearRegression()

coefficients_list = []
intercepts_list = []
scores = []
# Perform the KFold cross-validation
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', models)
])
# Initialize the linear regression model
# Fit the model on the training data

In [11]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
scores = cross_val_score(pipeline, Ames.drop(columns='SalePrice'), Ames['SalePrice'], scoring = 'r2', cv=kf, n_jobs = -1)

In [15]:
scores

array([0.86459614, 0.91413442, 0.92324174, 0.91555726, 0.91746021])

In [14]:
fold_scores = []
fold_num = 1
for train_index, test_index in kf.split(Ames):
    X_train, X_test = Ames.drop(columns='SalePrice').iloc[train_index], Ames.drop(columns='SalePrice').iloc[test_index]
    y_train, y_test = Ames['SalePrice'].iloc[train_index], Ames['SalePrice'].iloc[test_index]
    pipeline.fit(X_train, y_train)
    fold_score = pipeline.score(X_test, y_test)
    print(f'Score for Fold: {fold_num}', fold_score)
    fold_scores.append({
        'score':fold_score,
        'fold':fold_num})
    fold_num += 1

C:\Users\immed\Programming\Anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [12, 17, 25] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\immed\Programming\Anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [23, 33, 38, 41] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Score for Fold: 1 0.8645961418501235
Score for Fold: 2 0.9141344203197637


C:\Users\immed\Programming\Anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [12, 13, 35, 38] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\immed\Programming\Anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [9, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Score for Fold: 3 0.9232417402368964
Score for Fold: 4 0.9155572572163262
Score for Fold: 5 0.9174602102362468


C:\Users\immed\Programming\Anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [9, 12, 13, 22, 30, 31, 43] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [16]:
# pipeline.fit(X, y)
# # Store coefficients and intercepts
# coefficients_list.append(pipeline.named_steps['regressor'].coef_)
# intercepts_list.append(pipeline.named_steps['regressor'].intercept_)
# # Evaluate the model
# scores.append(pipeline.score(X, y))
# # Calculate the mean of scores, coefficients, and intercepts
# average_score = np.mean(scores)
# average_coefficients = np.mean(coefficients_list, axis=0)
# # Display the average R² score across all folds
# print(f"Mean CV R² Score of Combined Model: {average_score:.4f}")
# mean_intercept = np.mean(intercepts_list)
# print(f"Mean Y-intercept = {mean_intercept:.0f}")

#This is the entire data set, not being split, being trained and having the entire data set being tested. 

In [17]:
feature_names = []
# Adding electrical features (assuming it's still handled as a single feature post ordinal encoding)
feature_names.append('Electrical')
# Adding numeric features
feature_names.extend(numeric_features)
# Generating one-hot encoded feature names
onehot_features = pipeline.named_steps['preprocessor'].named_transformers_['nominal'].named_steps['onehot'].get_feature_names_out()
feature_names.extend(onehot_features)

278

In [124]:
feature_names = list(feature_names)# Create a DataFrame for the coefficients
df_coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Average Coefficient': average_coefficients
    }).sort_values(by='Average Coefficient').reset_index(drop=True)
# Display the DataFrame
print("Coefficients for Combined Model:")
print(df_coefficients)

ValueError: All arrays must be of the same length